# Can Synthetic Magnetic Field Data Reveal MRI-Induced Perturbations?

### A Computational Study Using the Vector Potential Method

*Author: [Your Name]*

## Introduction
In MRI safety research, translational forces on metallic objects — such as epicardial leads — are influenced by spatial gradients of the static magnetic field, \( B_0 \). Accurately reconstructing this field is essential when assessing whether an object might move during scanning. However, in real measurements, noise and uncertainty obscure the true field.

In this notebook, we ask:

> **How accurately can we reconstruct the magnetic field from a solenoid using synthetic 3D data, and how do measurement uncertainties affect our ability to detect field perturbations — like those caused by small metal implants during MRI?**

We will simulate a solenoid using the vector potential method, derive the magnetic field via the curl, and introduce synthetic noise to mimic real-world uncertainty. We then analyze the resulting field with statistical tools to assess detectability of potential perturbations.

---

## 1. Magnetic Vector Potential \( \mathbf{A} \) of a Solenoid
We start by simulating the 3D magnetic vector potential field \( \mathbf{A} \) for an ideal solenoid. This forms the foundation for computing the magnetic field.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.constants import mu_0

def generate_3d_grid(x_range, y_range, z_range, num_points=20):
    x = np.linspace(*x_range, num_points)
    y = np.linspace(*y_range, num_points)
    z = np.linspace(*z_range, num_points)
    return np.meshgrid(x, y, z, indexing="ij")

def vector_potential_solenoid(N, I, R, grid):
    X, Y, Z = grid
    rho = np.sqrt(X**2 + Y**2)
    A_phi = (mu_0 * N * I / (2 * np.pi)) * (1 / (rho + 1e-9))
    Ax = -A_phi * Y / (rho + 1e-9)
    Ay = A_phi * X / (rho + 1e-9)
    Az = np.zeros_like(Z)
    return Ax, Ay, Az

# Parameters
N, I, R = 1000, 5, 0.1
grid = generate_3d_grid((-0.2, 0.2), (-0.2, 0.2), (-0.5, 0.5))
Ax, Ay, Az = vector_potential_solenoid(N, I, R, grid)

## 2. Compute Magnetic Field \( \mathbf{B} = \nabla \times \mathbf{A} \)
Using finite differences, we compute the magnetic field from the curl of \( \mathbf{A} \).

In [ ]:
def compute_B_from_A(Ax, Ay, Az, grid_step):
    dAydz, dAydx = np.gradient(Ay, grid_step, axis=(2, 0))
    dAzdx, dAzdy = np.gradient(Az, grid_step, axis=(0, 1))
    dAxdz, dAxdy = np.gradient(Ax, grid_step, axis=(2, 1))
    Bx = dAydz - dAzdy
    By = dAzdx - dAxdz
    Bz = dAxdy - dAydx
    return Bx, By, Bz

Bx, By, Bz = compute_B_from_A(Ax, Ay, Az, 0.02)

## 3. Add Synthetic Noise
We introduce Gaussian noise to simulate measurement uncertainty in the magnetic field components.

In [ ]:
def add_noise(field, std_dev=0.02):
    return field + np.random.normal(0, std_dev, size=field.shape)

Bz_noisy = add_noise(Bz)

## 4. Chi-Square Test
We apply a \( \chi^2 \) test to compare the noisy \( B_z \) data with the true field.

In [ ]:
from scipy.stats import chisquare

def chi2_test_3d(observed, expected):
    return chisquare(observed.flatten(), expected.flatten())

chi2_stat, p_value = chi2_test_3d(Bz_noisy, Bz)
print(f'Chi^2 statistic: {chi2_stat:.2f}, p-value: {p_value:.4f}')

## 5. Conclusion
This study demonstrates that 3D synthetic data from a solenoid can be analyzed statistically to evaluate detectability of deviations from the ideal field.
Future extensions will include adding a static metal object to simulate field distortions and estimating induced forces for displacement analysis.